In [ ]:
import pandas as pd
import numpy as np
import pickle

from joblib import Parallel, delayed
from fbprophet import Prophet

## Params

In [ ]:
IS_EVAL = True
DATA_PATH = '../data/'

if IS_EVAL:
    PERIOD_LABEL = 'evaluation'
else:
    PERIOD_LABEL = 'validation'

## Load needed data

In [ ]:
prophet_df = pd.read_pickle(DATA_PATH + 'refined/prophet_df_' + PERIOD_LABEL + '.pkl')
prophet_params = pd.read_csv(DATA_PATH + 'external/params_prophet_store_dpt_' + PERIOD_LABEL + '.csv')
sample_submission = pd.read_csv(DATA_PATH + 'raw/sample_submission.csv')

## Forecast

In [ ]:
def forecast_prophet(store_id, dept_id):
    
    # Reduce df & params on current 
    df = prophet_df[(prophet_df['store_id'] == store_id) & (prophet_df['dept_id'] == dept_id)].copy()
    params = prophet_params.loc[(prophet_params['store_id'] == store_id) & 
                                (prophet_params['dept_id'] == dept_id), 'params'].values[0]
    params = eval(params) # String to dict
    
    # Define model
    m = Prophet(
            yearly_seasonality=False,
            weekly_seasonality=False,
            daily_seasonality=False,
            uncertainty_samples=False,
            changepoint_range=params['changepoint_range'],
            changepoint_prior_scale=params['changepoint_prior_scale'],
            holidays_prior_scale=params['holidays_prior_scale'],
            seasonality_mode=params['seasonality_mode']
        )
        
    m.add_seasonality(
        name='yearly', 
        period=365.25, 
        fourier_order=params['yearly_order'],
        prior_scale=params['yearly_prior_scale']
    )
    
    m.add_seasonality(
        name='monthly', 
        period=365.25/12, 
        fourier_order=params['monthly_order'],
        prior_scale=params['monthly_prior_scale']
    )
    
    m.add_seasonality(
        name='weekly', 
        period=7, 
        fourier_order=params['weekly_order'],
        prior_scale=params['weekly_prior_scale']
    )
    
    # Add holidays/regressor
    m.add_country_holidays(country_name='US')
    
    m.add_regressor('snap', mode=params['snap_mode'])
        
    m.add_regressor('price', mode=params['price_mode'])
    
    m.add_regressor('dom', mode=params['dom_mode'])
    
    # Fit 
    m.fit(df.dropna(subset=['y'])) # drop pred period
    
    # Predict
    future = m.make_future_dataframe(periods=28, freq='D', include_history=False)
    
    future['snap'] = df['snap'].values[-28:]
    future['price'] = df['price'].values[-28:]
    future['dom'] = df['dom'].values[-28:]
    
    fcst = m.predict(future)
    
    # Add ids & d to the output
    fcst['store_id'] = store_id
    fcst['dept_id'] = dept_id
    fcst['d'] = df['d'].values[-28:]
    
    # Round forecast
    fcst['yhat'] = fcst['yhat'].round().astype(int)
    fcst.loc[fcst['yhat'] < 0, 'yhat'] = 0
        
    return fcst[['store_id', 'dept_id', 'd', 'yhat']]

In [ ]:
ts_ids = prophet_df[['store_id', 'dept_id']].drop_duplicates().reset_index(drop=True)

res = Parallel(n_jobs=-1, verbose=1)\
              (delayed(forecast_prophet)\
              (row['store_id'], row['dept_id']) for _, row in ts_ids.iterrows())

In [ ]:
# retrieve forecasts
forecast_df = pd.concat(res)
forecast_df.head()

## Format and save as external data

In [ ]:
forecast_df = pd.pivot_table(forecast_df, index=['store_id', 'dept_id'], columns='d', values='yhat')
forecast_df.columns = ['F' + str(int(c) + 1) for c in range(28)]
forecast_df = forecast_df.reset_index()
forecast_df.head()

In [ ]:
forecast_df.to_csv(DATA_PATH + 'external/forecast_prophet_store_dpt_' + PERIOD_LABEL + '.csv', index=False)